In [ ]:
import json
import boto3
from decimal import Decimal

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table("VisitorCountTable")  # Your actual table name

def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)

def lambda_handler(event, context):
    print("Received Event:", json.dumps(event, indent=2))

    try:
        method = event.get("httpMethod") or event.get("requestContext", {}).get("http", {}).get("method")

        if method == "POST":
            response = table.update_item(
                Key={'id': 'visitorCount'},
                UpdateExpression="SET #c = if_not_exists(#c, :start) + :inc",
                ExpressionAttributeNames={"#c": "count"},
                ExpressionAttributeValues={":inc": 1, ":start": 0},
                ReturnValues="UPDATED_NEW"
            )

            updated_count = response['Attributes']['count']
            print(f"Updated visitor count: {updated_count}")

            return {
                'statusCode': 200,
                'headers': {
                    'Content-Type': 'application/json',
                    'Access-Control-Allow-Origin': '*'
                },
                'body': json.dumps({'visitorCount': updated_count}, default=decimal_default)
            }

        elif method == "GET":
            response = table.get_item(Key={'id': 'visitorCount'})
            count = response.get('Item', {}).get('count', 0)

            return {
                'statusCode': 200,
                'headers': {
                    'Content-Type': 'application/json',
                    'Access-Control-Allow-Origin': '*'
                },
                'body': json.dumps({'visitorCount': count}, default=decimal_default)
            }

        return {'statusCode': 405, 'body': json.dumps({'error': 'Method Not Allowed'})}

    except Exception as e:
        print(f"Error: {str(e)}")
        return {'statusCode': 500, 'body': json.dumps({'error': str(e)})}
